In [1]:
## This pipeline generates the seroreactive peptide set of 9927 peptides used in all further analyses 
# There a few files created in the process that could be used for running any of these steps in between without having to start from the top
## The final output file that has the set of 9927 seroreactive squences with information on whether it is seroreactive or not in a given sample (binary coding - 1/0) is 'HITS_Round2_Pfonly_250kfil_techclean_3zscorefil_5patientsfil.csv'


### Filtering of data 

### Getting only Round2 reads 
### Discarding samples with <250,000 tooal reads
### Discarding samples with only 1 tech replicate remaining after filtering by read count 
### Calculate reads per 500,000 - RP5K - this threshold chosen because most samplees have atleast 500,000 reads and so we can make use of that depth 
import pandas as pd 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np 

df_all = pd.read_csv("Gsnap_pairedend_alignment_raw_readcounts.csv",header=0,index_col = 'peptide') ##peptide is the index
df = df_all.filter(regex='Round2') ###Round2 
total = df.sum(axis=0, numeric_only = True) ##get the total counts across all rows for each column 

### Choose the read depth normalization metric based on the read depth of the dataset
## In the present dataset, most of the samples have at least 500,000 reads. 
## So reads per 500,000 has been used for this dataset 

df_rpk = df.div(0.00001*total,axis=1) ##calculate rpk (reads per 100,000) 
df_rpk = df_rpk.mul(5,axis=1)##calculate rpk (reads per 500,000)  

print ("Filtering the dataset")
print ("The number of total peptides is" + str(df.shape[0]))
print ("The number of samples in Round 2 " + str(df.shape[1])) 

## Remove samples with less than 250,000 raw reads  

df_rpk_countfil = df_rpk[df_rpk.columns.drop(total[total<250000].index)] 

print ("The number of samples remaining after filteering for <250,000 reads " + str(df_rpk_countfil.shape[1])) 

## identify samples where only 1 technical replicate is present 

col_name = []
for col in df_rpk_countfil.columns:### loop through column names and collect the first two parts in a list and then count occurence freq in the list 
    name = '_'.join(col.split("_")[0:2]) ## get the sample name that would be common in the other df as well
    col_name.append(name) ## collect these names to the rerun list 

from collections import Counter

freq = dict(Counter(col_name)) ### Get frequency of items in a list as a dictionary 
print ("The samples with only 1 tech replicate are ")
l = [k for k, v in freq.items() if v == 1] ### Get sammples who have only 1 replicate         
print (l)
### remove samples with only 1 replicate 

to_remove = '|'.join(l)
df_rpk_clean = df_rpk_countfil.drop(df_rpk_countfil.filter(regex = to_remove,axis=1),axis=1)
print ("The number of sample lefte after removing single replicates " + str(df_rpk_clean.shape[1]))### Saving the df_rpk_clean df to a csv file
#df_rpk_clean.to_csv('Round2_RP5K_allpeptide_250kfilter_techclean.csv')




Filtering the dataset
The number of total peptides is238068
The number of samples in Round 2 768
The number of samples remaining after filteering for <250,000 reads 734
The samples with only 1 tech replicate are 
['Plate1_HC017', 'Plate2_GFAP', 'Plate2_HC035', 'Plate4_GFAP', 'Plate4_HC011', 'Plate4_HC044', 'Plate4_HC069', 'Plate4_HC070', 'Plate4_HC080', 'Plate4_HC084']
The number of sample lefte after removing single replicates 724


In [43]:
### Collecting Pf Only peptides 


### Dropping viral and human peptides 

df_PfOnly = df_rpk_clean.drop(list(df_rpk_clean.filter(regex = r'(virus|toxin|sapiens)',axis=0).index))

print ("The number of Pf + Anopheles peptide is " + str(df_PfOnly.shape[0]))

## Renormalizing RP5K values within the reduced dataset
total_Pf = df_PfOnly.sum(axis=0, numeric_only = True) ##get the total of the remaining rows for each column 
df_PfOnly_rpk = df_PfOnly.div(0.000002*total_Pf,axis=1) ##calculate new rp5k for PF only peptides 

df_PfOnly_rpk.to_csv('Round2_RP5K_PfOnlypeptide_250kfilter_techclean.csv')




The number of Pf + Anopheles peptide is 230556


In [45]:
### Calculating Z scores by Normal distribution for Pf only peptides and using a threshold of 3 and cmin of 5 to call hits 
### A peptide must be a hit in >75% of the replicates to be called a hit in a given sample 

""" detecting hits based on the healthy distribution
1. copy the rpk file into a new one 
2. subtract all rpks for a given peptide with respective healthy mean
3. Divide the above number with the healthy std dev for the peptide 
4. This number is the Z score 
5. If Z-score is >=3, call it a hit and give it a value of 1 in the new hit df. If not, give a value of 0. 
6. Also, store the Z-score in a new df. 
"""

df_PfOnly_rpk = pd.read_csv('Round2_RP5K_PfOnlypeptide_250kfilter_techclean.csv',header = 0, index_col = 'peptide')

df_PfOnly_rpk_Healthy = df_PfOnly_rpk.filter(regex = r'(HC|healthy|Healthy)',axis=1) ##collect the Healthy samples 

healthy_mean = df_PfOnly_rpk_Healthy.mean(axis=1,numeric_only=True) ##mean of healthy sampels for each peptide
healthy_SD = df_PfOnly_rpk_Healthy.std(axis=1,numeric_only=True) ##sd of healthy sampels for each peptide
healthy_SD.where(healthy_SD>1, 1, inplace=True) ###replace any SD <1 with 1 for ease in division later on. 



df_PfOnly_zscore = df_PfOnly_rpk.copy()

df_PfOnly_zscore = df_PfOnly_zscore.sub(healthy_mean,axis=0)  ### sample rpk - mean rpk
df_PfOnly_zscore = df_PfOnly_zscore.div(healthy_SD,axis=0) ### sample - mean divided by SD 

### Setting a threshold of Zscore of 3 for selecting as hits 
### look at technical replicates. Only if atleast 75% of the tech reps (3/4,4/5,5/6,2/2,3/3) call it a hit will it be considered a hit in that sample 


### Using PfOnly reads 

df_PfOnly_zscore3 = df_PfOnly_zscore.copy() ### copying zscores to a new df 
df_PfOnly_zscore3.where(df_PfOnly_zscore3>=3, 0, inplace= True)
df_PfOnly_zscore3.where((df_PfOnly_zscore3<3), 1, inplace= True)

"""
1. Transpose the table so that column names are indices
2. Split the index names and take only the common part before the Tech part 
3. Group technical replicates for each sample and take the mean 
4. If the mean is >= 0.75, call it a hit and give it a score of 1. If not, score is zero
"""

df_tech = df_PfOnly_zscore3.T ## transpose the matrix so that sample names are index names now 
df_tech['new_index'] = df_tech.index.to_series().str.split("_").str[1:2].str.join("_")
rep = df_tech.groupby('new_index').mean()
rep.where(rep>=0.75, 0, inplace= True)
rep.where(rep<0.75, 1, inplace= True)  
df_PfOnly_zscore3_hits = rep.T
df_PfOnly_zscore3_hits = df_PfOnly_zscore3_hits.loc[df_PfOnly_zscore3_hits.sum(axis=1)!=0]  ### Only consider those hits where reps are both hits 


print("Number of peptides hits (>z-score of 3) in atleast 1 patient/healthy sample (both replicates) is " + str(df_PfOnly_zscore3_hits.shape[0]))

Number of peptides hits (>z-score of 3) in atleast 1 patient/healthy sample (both replicates) is 84363


In [47]:
#### Setting 1, 3, 5, 8 patients as a threshold for selecting Malaria specific hits and boxing them in 3 categories. 

### 1. Collecting the number of healthys and patients that are HITS for each peptide
df_aggregate_hit = df_PfOnly_zscore3_hits.copy()
df_aggregate_Healthy = df_aggregate_hit.filter(regex = r'(HC|healthy|Healthy)',axis=1) ##collect the Healthy samples 
healthy_col = df_aggregate_Healthy.columns
df_aggregate_Patient = df_aggregate_hit.drop(list(df_aggregate_hit.filter(regex = r'(HC|Healthy|gfap|Canary|GFAP|4F42|canary)',axis=1).columns),axis =1)
patient_col = df_aggregate_Patient.columns

healthyhitsum = df_aggregate_hit[healthy_col].sum(axis=1)
patienthitsum = df_aggregate_hit[patient_col].sum(axis=1)

hit = pd.concat([healthyhitsum,patienthitsum],axis=1)
hit.columns = ['healthy_R2','patient_R2']

### Calculate mean zscore of replicates for the hits 
z_tech = df_PfOnly_zscore.T ## transpose the matrix so that sample names are index names now 
z_tech['new_index'] = z_tech.index.to_series().str.split("_").str[1:2].str.join("_") ###combining data for the healthys from plates 3 and 4  
z_mean = z_tech.groupby('new_index').mean()
# z_SD = z_tech.groupby('new_index').std()

patient_5 = hit[hit.patient_R2>4]

df_PfOnly_zscore3_hits.loc[patient_5.index].to_csv('HITS_Round2_Pfonly_250kfil_techclean_3zscorefil_5patientsfil.csv')

print ("The number of hits with 3zscore and 5 patient threshold is " + str(patient_5.shape[0]))

z_mean.T.loc[patient_5.index].to_csv('HITS_meanZscore_Round2_Pfonly_250kfil_techclean_3zscorefil_5patientsfil.csv')


The number of hits with 3zscore and 1 patient threshold is 63773
The number of hits with 3zscore and 3 patient threshold is 20785
The number of hits with 3zscore and 5 patient threshold is 9927
The number of hits with 3zscore and 8 patient threshold is 5018
